In [149]:
import geopandas as gpd
from requests import Request
from owslib.wfs import WebFeatureService
from IPython.display import GeoJSON

In [127]:
url = "https://pubgeo.zwemwater.nl/geoserver/zwr_public/wfs"
wfs = WebFeatureService(url=url)

In [128]:
def load_wfs_layer(layer_name):
    params = dict(service='WFS', version="2.0.0", request='GetFeature',
      typeName=layer_name, outputFormat='json', srsName="EPSG:4326")
    wfs_request_url = Request('GET', url, params=params).prepare().url
    return gpd.read_file(wfs_request_url)

# Data

### Zwemplekken

In [129]:
zwemplekken = load_wfs_layer("zwr_public:zwemplekken")
zwemplekken = zwemplekken.drop(columns=['id', 'datum']).drop_duplicates(subset=["zwemwaterlocatie_id"]).set_index('key_id')
zwemplekken

,korte_naam,naam,status,zwemwaterlocatie_id,geometry
key_id,,,,,
6187300,Voorveldse Polder,Voorveldse Polder,NEGATIEF_ZWEMADVIES,1185,POINT (5.1522 52.1005)
2316,'t Kleine Zeetje,Recreatiepark 't Kleine Zeetje,NEGATIEF_ZWEMADVIES,1595,POINT (5.3762 52.2602)
1954,Plas Vrijenburgbos,Plas Vrijenburgbos,NEGATIEF_ZWEMADVIES,1233,POINT (4.5003 51.868)
1962,Kralinger Esch,Kralinger Esch,NEGATIEF_ZWEMADVIES,1241,POINT (4.5253 51.9138)
2307,De Oude Pol,De Oude Pol,NEGATIEF_ZWEMADVIES,1586,POINT (5.7259 52.3925)
...,...,...,...,...,...
8184932,'t Gorsselaar,Bussloo 't Gorsselaar,goed,8184902,POINT (6.0963 52.2021)
8184934,De Kuiter,Bussloo De Kuiter (Naturistenstrand),goed,8184903,POINT (6.0998 52.1959)
8186500,Aquapark,Kempervennen Aquapark,goed,8186460,POINT (5.4247 51.3272)


In [130]:
zwemplekken_details = load_wfs_layer("zwr_public:zwemplekken_details")
zwemplekken_details = zwemplekken_details.drop(columns=["id", "info_id", "geometry", "datum"])
zwemplekken_details

,adr_huisletter,adr_huisnummer,adr_huisnummertoevoeging,adr_postcode,adr_straat,adr_woonplaats,info_filename,info_length,info_mime_type,key_id,...,org_straat,org_telefoon,org_website,org_woonplaats,status,taal,tekst,toegang,website,zwemwaterlocatie_id
0,None,NaN,None,None,De Dreef,Heerenveen,FR-043-001 De Heide.jpg,356023.0,image/jpeg,1676,...,,0513-617 617,http://www.heerenveen.nl,Heerenveen,goed,nl,"Het recreatiegebied “De Heide”, ligt aan de zu...",OPENBAAR ONBETAALD,None,955
1,,NaN,,None,Spokedam,Noordwolde,FR-046-001 Spoekeplas.jpg,297473.0,image/pjpeg,1677,...,,050 707 4444,http://www.staatsbosbeheer.nl,Groningen,goed,nl,Het zwemwater Spoekeplas ligt ten zuidwesten v...,OPENBAAR ONBETAALD,None,956
2,None,NaN,None,None,Canada,Appelscha,Canadameer Appelscha.png,212248.0,image/png,1678,...,,050 707 4444,http://www.staatsbosbeheer.nl,Groningen,goed,nl,Het Canadameer (ook wel Aekingermeer genoemd) ...,OPENBAAR ONBETAALD,None,957
3,None,NaN,None,None,Kleasterkampen,Smalle Ee,FR-028-001 Smeliester SaÌ‚n.jpg,210638.0,image/pjpeg,1680,...,,0512-581234,http://www.smallingerland.nl,Drachten,goed,nl,Het zwemwater ligt aan de zuidzijde van het ge...,OPENBAAR ONBETAALD,None,959
4,None,NaN,None,None,Bildreed,Rottevalle,FR-027-001 De Leien.jpg,209646.0,image/pjpeg,1681,...,,050 707 4444,http://www.staatsbosbeheer.nl,Groningen,WAARSCHUWING,nl,Het zwemwater ligt enkele kilometers ten noord...,OPENBAAR ONBETAALD,None,960
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
950,None,NaN,None,None,Kneuterstraat,Wilp,GLD-004-007 BUSSLOO GORSSELAAR 2020.jpg,2995096.0,image/jpeg,8184932,...,Westervoortsedijk 73-LB,026-3848800,http://www.leisurelands.nl,Arnhem,goed,nl,De recreatieplas Bussloo ligt temidden van de ...,Openbaar betaald parkeren voor auto's,http://www.leisurelands.nl/,8184902
951,None,NaN,None,None,Bloemenksweg,Wilp,GLD-004-013 BUSSLOO DE KUITER 2020.jpg,2981855.0,image/jpeg,8184934,...,Westervoortsedijk 73-LB,026-3848800,http://www.leisurelands.nl,Arnhem,goed,nl,De recreatieplas Bussloo ligt temidden van de ...,"Openbaar, betaald parkeren voor auto's",http://www.leisurelands.nl/,8184903
952,None,8.0,None,5563VB,Kempervennendreef,Westerhoven,None,NaN,None,8186500,...,Bosscheweg,(0411) 618 618,http://www.dommel.nl/,Boxtel,goed,nl,zwemlocatie Aquapark is ingericht als een drij...,Openbaar toegankelijk via kaartjesverkoop,None,8186460
953,None,35.0,None,3263RA,Randweg,Oud-Beijerland,None,NaN,None,8192100,...,Sportlaan,088-6473647,https://www.gemeentehw.nl/,Maasdam,goed,nl,De zwemlocatie heeft een zwembaai gelegen aan ...,None,None,8192060


In [131]:
base_data = zwemplekken_details.copy()
base_data = base_data[["zwemwaterlocatie_id", "naam", "korte_naam", "status", "tekst", "info_filename", "website"]]
base_data

,zwemwaterlocatie_id,naam,korte_naam,status,tekst,info_filename,website
0,955,De Heide,Heide,goed,"Het recreatiegebied “De Heide”, ligt aan de zu...",FR-043-001 De Heide.jpg,None
1,956,Spoekeplas,Spoekeplas,goed,Het zwemwater Spoekeplas ligt ten zuidwesten v...,FR-046-001 Spoekeplas.jpg,None
2,957,Canadameer,Canadameer,goed,Het Canadameer (ook wel Aekingermeer genoemd) ...,Canadameer Appelscha.png,None
3,959,Smeliester Sân,Smeliester Sân,goed,Het zwemwater ligt aan de zuidzijde van het ge...,FR-028-001 Smeliester SaÌ‚n.jpg,None
4,960,De Leien,De Leien,WAARSCHUWING,Het zwemwater ligt enkele kilometers ten noord...,FR-027-001 De Leien.jpg,None
...,...,...,...,...,...,...,...
950,8184902,Bussloo 't Gorsselaar,'t Gorsselaar,goed,De recreatieplas Bussloo ligt temidden van de ...,GLD-004-007 BUSSLOO GORSSELAAR 2020.jpg,http://www.leisurelands.nl/
951,8184903,Bussloo De Kuiter (Naturistenstrand),De Kuiter,goed,De recreatieplas Bussloo ligt temidden van de ...,GLD-004-013 BUSSLOO DE KUITER 2020.jpg,http://www.leisurelands.nl/
952,8186460,Kempervennen Aquapark,Aquapark,goed,zwemlocatie Aquapark is ingericht als een drij...,None,None
953,8192060,Oude Tol,Oude Tol,goed,De zwemlocatie heeft een zwembaai gelegen aan ...,None,None


In [132]:
zwemplekdata = zwemplekken.merge(base_data, left_on="zwemwaterlocatie_id", right_on="zwemwaterlocatie_id", how="left")
zwemplekdata['lat'] = zwemplekdata['geometry'].y
zwemplekdata['lon'] = zwemplekdata['geometry'].x
zwemplekdata = zwemplekdata.drop(columns=["korte_naam_x", "naam_x", "status_x", "geometry"])
zwemplekdata = zwemplekdata.rename(columns={
    "naam_y": "naam",
    "korte_naam_y": "korte_naam",
    "status_y": "status"
})
zwemplekdata.sort_values(by=["naam"], inplace=True)
zwemplekdata

,zwemwaterlocatie_id,naam,korte_naam,status,tekst,info_filename,website,lat,lon
820,1323,Engelermeer Oost,Engelermeer Oost,goed,Het Engelermeer is een serie kunstmatige plass...,NB-031-001 Engelermeer Oost.jpg,None,51.7046,5.2461
797,1259,'T Hoefsven,'T Hoefsven,goed,De locatie ligt ten zuidoosten van Waalwijk. E...,NB-063-001 't Hoefsven.jpg,None,51.6761,5.0942
338,1341,'T Nije Hemelriek,'T Nije Hemelriek,goed,De zwemlocatie “’t Nije Hemelriek” is gelegen ...,Recreatiezone_'t_Nije_Hemelriek.jpg,None,52.9725,6.7556
827,1315,'T Schaartven,'T Schaartven,goed,"‘t Schaartven is een voormalige zandwinplas, g...",None,None,51.5735,5.9749
341,1344,'T Veenmeer,'T Veenmeer,goed,De zwemlocatie ‟t Veenmeer is gelegen in Tynaa...,Recreatiezone_'t_Veenmeer.jpg,None,53.0829,6.6303
...,...,...,...,...,...,...,...,...,...
63,7941840,strand Kortgene Paardekreek,Paardekreek,WAARSCHUWING,Door camping de Paardekreek aangelegd zandstra...,None,None,51.5496,3.8096
921,7356682,strand zoutkamp,None,goed,strand zoutkamp in onderzoek,None,None,53.3388,6.2973
903,6837320,strand zuid Oldambtmeer,strand zuid,goed,strand zuid oldambtmeer,Recreatiezone Zuid Blauwe Stad kopie-1.jpg,None,53.1651,7.0372
902,6997562,strandje Brouwershaven,Brouwershaven,goed,Strandje naast jachthaven van Brouwershaven,None,None,51.7315,3.9154


### EU statussen

In [133]:
eu_designations = load_wfs_layer("zwr_public:eustatussen")
eu_designations = eu_designations[eu_designations['jaar'] == 2023].drop(columns=['type_eu_status_id', 'geometry', 'omschrijving', 'id'])
eu_designations

,code,key_id,jaar,zwemwaterlocatie_id
7010,1,8135490,2023,1267
7011,1,8135491,2023,1268
7012,1,8135492,2023,1270
7013,1,8135493,2023,1271
7014,1,8135494,2023,1273
...,...,...,...,...
7751,4,8167548,2023,1112
7752,4,8167553,2023,1118
7753,3,8167559,2023,1124
7754,4,8167555,2023,1120


In [134]:
eu_current_designations = eu_designations.query("jaar == 2023")
eu_current_designations = eu_current_designations.drop(columns=["key_id", "jaar"])
eu_current_designations
data = zwemplekdata.merge(eu_current_designations, left_on="zwemwaterlocatie_id", right_on="zwemwaterlocatie_id", how="left")
data.fillna({'code': 0}, inplace=True)
data

,zwemwaterlocatie_id,naam,korte_naam,status,tekst,info_filename,website,lat,lon,code
0,1323,Engelermeer Oost,Engelermeer Oost,goed,Het Engelermeer is een serie kunstmatige plass...,NB-031-001 Engelermeer Oost.jpg,None,51.7046,5.2461,1
1,1259,'T Hoefsven,'T Hoefsven,goed,De locatie ligt ten zuidoosten van Waalwijk. E...,NB-063-001 't Hoefsven.jpg,None,51.6761,5.0942,1
2,1341,'T Nije Hemelriek,'T Nije Hemelriek,goed,De zwemlocatie “’t Nije Hemelriek” is gelegen ...,Recreatiezone_'t_Nije_Hemelriek.jpg,None,52.9725,6.7556,1
3,1315,'T Schaartven,'T Schaartven,goed,"‘t Schaartven is een voormalige zandwinplas, g...",None,None,51.5735,5.9749,1
4,1344,'T Veenmeer,'T Veenmeer,goed,De zwemlocatie ‟t Veenmeer is gelegen in Tynaa...,Recreatiezone_'t_Veenmeer.jpg,None,53.0829,6.6303,1
...,...,...,...,...,...,...,...,...,...,...
950,7941840,strand Kortgene Paardekreek,Paardekreek,WAARSCHUWING,Door camping de Paardekreek aangelegd zandstra...,None,None,51.5496,3.8096,1
951,7356682,strand zoutkamp,None,goed,strand zoutkamp in onderzoek,None,None,53.3388,6.2973,1
952,6837320,strand zuid Oldambtmeer,strand zuid,goed,strand zuid oldambtmeer,Recreatiezone Zuid Blauwe Stad kopie-1.jpg,None,53.1651,7.0372,2
953,6997562,strandje Brouwershaven,Brouwershaven,goed,Strandje naast jachthaven van Brouwershaven,None,None,51.7315,3.9154,1


### Metingen

In [135]:
measurements = load_wfs_layer("zwr_public:resultaatsen")
measurements = measurements.drop(columns=['id', 'datum_geplande_monstername', 'monitoring_datum_id', 'monitoring_plan_id', 'monster_id', 'type_object_id', 'geometry'])
measurements = measurements.sort_values(["zwemwaterlocatie_id", "object_begin_tijd", "type_object_code"])
measurements = measurements.drop_duplicates(subset=["zwemwaterlocatie_id", "type_object_code"], keep="last")
measurements


,numerieke_waarde,object_begin_tijd,key_id,type_object_code,zwemwaterlocatie_id
4213,90,2024-07-22 07:05:00+00:00,8321908,E_COLI,955
8788,15,2024-07-22 07:05:00+00:00,8321909,INTTNLETRCCN,955
4362,20,2024-07-29 09:03:00+00:00,8328070,E_COLI,956
8928,15,2024-07-29 09:03:00+00:00,8328071,INTTNLETRCCN,956
4363,15,2024-07-29 07:57:00+00:00,8328102,E_COLI,957
...,...,...,...,...,...
8858,15,2024-07-23 11:30:00+00:00,8325003,INTTNLETRCCN,8186460
4561,45,2024-07-30 08:48:00+00:00,8333554,E_COLI,8192060
9127,15,2024-07-30 08:48:00+00:00,8333556,INTTNLETRCCN,8192060
4379,20,2024-07-29 06:20:00+00:00,8328607,E_COLI,8194200


In [136]:
measure = measurements.pivot_table(values='numerieke_waarde', index='zwemwaterlocatie_id', columns='type_object_code')
measure

type_object_code,E_COLI,INTTNLETRCCN
zwemwaterlocatie_id,,
955,90.0,15.0
956,20.0,15.0
957,15.0,15.0
959,50.0,60.0
960,980.0,140.0
...,...,...
8184902,15.0,15.0
8184903,144.0,15.0
8186460,15.0,15.0


In [137]:
data = data.merge(measure, on="zwemwaterlocatie_id", how="left")
data

,zwemwaterlocatie_id,naam,korte_naam,status,tekst,info_filename,website,lat,lon,code,E_COLI,INTTNLETRCCN
0,1323,Engelermeer Oost,Engelermeer Oost,goed,Het Engelermeer is een serie kunstmatige plass...,NB-031-001 Engelermeer Oost.jpg,None,51.7046,5.2461,1,61.0,30.0
1,1259,'T Hoefsven,'T Hoefsven,goed,De locatie ligt ten zuidoosten van Waalwijk. E...,NB-063-001 't Hoefsven.jpg,None,51.6761,5.0942,1,15.0,46.0
2,1341,'T Nije Hemelriek,'T Nije Hemelriek,goed,De zwemlocatie “’t Nije Hemelriek” is gelegen ...,Recreatiezone_'t_Nije_Hemelriek.jpg,None,52.9725,6.7556,1,15.0,15.0
3,1315,'T Schaartven,'T Schaartven,goed,"‘t Schaartven is een voormalige zandwinplas, g...",None,None,51.5735,5.9749,1,15.0,15.0
4,1344,'T Veenmeer,'T Veenmeer,goed,De zwemlocatie ‟t Veenmeer is gelegen in Tynaa...,Recreatiezone_'t_Veenmeer.jpg,None,53.0829,6.6303,1,15.0,15.0
...,...,...,...,...,...,...,...,...,...,...,...,...
950,7941840,strand Kortgene Paardekreek,Paardekreek,WAARSCHUWING,Door camping de Paardekreek aangelegd zandstra...,None,None,51.5496,3.8096,1,15.0,15.0
951,7356682,strand zoutkamp,None,goed,strand zoutkamp in onderzoek,None,None,53.3388,6.2973,1,161.0,15.0
952,6837320,strand zuid Oldambtmeer,strand zuid,goed,strand zuid oldambtmeer,Recreatiezone Zuid Blauwe Stad kopie-1.jpg,None,53.1651,7.0372,2,253.0,46.0
953,6997562,strandje Brouwershaven,Brouwershaven,goed,Strandje naast jachthaven van Brouwershaven,None,None,51.7315,3.9154,1,77.0,15.0


### Voorzieningen

In [138]:
zwemplek_voorziening = load_wfs_layer("zwr_public:zwemplek_voorziening")
zwemplek_voorziening

,id,aanvulling,code,groep,omschrijving,taal,key_id,voorziening_type_id,zwemplek_id,zwemwaterlocatie_id,geometry
0,zwemplek_voorziening.fid--78231000_191126d659e...,None,5943907,GENERIEK,Infobord,None,597838,5943907,1811,1090,"POLYGON ((4.9613 52.2491, 4.9615 52.2492, 4.96..."
1,zwemplek_voorziening.fid--78231000_191126d659e...,None,5943892,HYGIENE,Afvalbakken,None,599138,5943892,1935,1214,"POLYGON ((4.2478 51.8493, 4.2484 51.8496, 4.24..."
2,zwemplek_voorziening.fid--78231000_191126d659e...,None,5943916,RECREATIE,Speelweide,None,6802511,5943916,6566150,6502091,POINT (5.7796 51.519)
3,zwemplek_voorziening.fid--78231000_191126d659e...,EHBO post,5943900,ONBEPERKT,EHBO,nl,594394,5943900,1712,991,"POLYGON ((6.128 52.5405, 6.1282 52.5406, 6.129..."
4,zwemplek_voorziening.fid--78231000_191126d659e...,None,5943915,RECREATIE,Speeltoestellen,None,597839,5943915,1811,1090,"POLYGON ((4.9613 52.2491, 4.9615 52.2492, 4.96..."
...,...,...,...,...,...,...,...,...,...,...,...
10998,zwemplek_voorziening.fid--78231000_191126d659e...,None,5943929,RECREATIE,Zandstrand,None,8200653,5943929,7714058,1675,"POLYGON ((4.5976 52.5286, 4.5986 52.5285, 4.59..."
10999,zwemplek_voorziening.fid--78231000_191126d659e...,None,5943929,RECREATIE,Zandstrand,None,8200654,5943929,2396,1675,"POLYGON ((4.5958 52.5236, 4.5957 52.5231, 4.59..."
11000,zwemplek_voorziening.fid--78231000_191126d659e...,None,5943929,RECREATIE,Zandstrand,None,8200655,5943929,7714057,1675,"POLYGON ((4.5863 52.4989, 4.5876 52.4986, 4.58..."
11001,zwemplek_voorziening.fid--78231000_191126d659e...,None,5943907,GENERIEK,Infobord,None,8273310,5943907,8011970,1115,"POLYGON ((4.7642 52.6814, 4.7641 52.681, 4.764..."


In [139]:
voorzieningen = zwemplek_voorziening.groupby('zwemwaterlocatie_id')['voorziening_type_id'].agg(list).reset_index()
voorzieningen

,zwemwaterlocatie_id,voorziening_type_id
0,955,"[5943905, 5943906, 5943913, 5943913, 5943902, ..."
1,956,"[5943905, 5943906, 5943913, 5943913, 5943902, ..."
2,957,"[5943913, 5943930, 5943929, 5943907, 5943911, ..."
3,959,"[5943913, 5943913, 5943902, 5943930, 5943929, ..."
4,960,"[5943906, 5943913, 5943913, 5943902, 5943930, ..."
...,...,...
714,8184901,"[5943891, 5943892, 5943916, 5943907, 5943929, ..."
715,8184902,"[5943892, 5943916, 5943913, 5943907, 5943929, ..."
716,8184903,"[5943891, 5943898, 5943892, 5943916, 5943913, ..."
717,8186460,"[5943928, 5943892, 5943915, 5943908, 5943917]"


In [140]:
data = data.merge(voorzieningen, on="zwemwaterlocatie_id", how="left")
data.rename(columns={
    "naam": "name",
    "korte_naam": "alternate_name",
    "status": "current_status",
    "tekst": "description",
    "info_filename": "photos",
    "code": "eu_designation",
    "E_COLI": "e_coli",
    "INTTNLETRCCN": "int_ent",
    "voorziening_type_id": "voorzieningen"
}, inplace=True)
data.set_index("zwemwaterlocatie_id", inplace=True)
data['name'] = data['name']\
    .str.strip()\
    .str.title()\
    .str.replace("^Rcn", "RCN", regex=True)\
    .str.replace("^T ", "'t ", regex=True)\
    .str.replace("['|`]T ", "'t ", regex=True)\
    .str.replace("['|`]S", "'s", regex=True)
data['alternate_name'] = data['alternate_name']\
    .str.strip()\
    .str.title()\
    .str.replace("^Rcn", "RCN", regex=True)\
    .str.replace("^T ", "'t ", regex=True)\
    .str.replace("['|`]T ", "'t ", regex=True)\
    .str.replace("['|`]S", "'s", regex=True)
data

,name,alternate_name,current_status,description,photos,website,lat,lon,eu_designation,e_coli,int_ent,voorzieningen
zwemwaterlocatie_id,,,,,,,,,,,,
1323,Engelermeer Oost,Engelermeer Oost,goed,Het Engelermeer is een serie kunstmatige plass...,NB-031-001 Engelermeer Oost.jpg,None,51.7046,5.2461,1,61.0,30.0,"[5943902, 5943930, 5943929, 5943896, 5943923, ..."
1259,'t Hoefsven,'t Hoefsven,goed,De locatie ligt ten zuidoosten van Waalwijk. E...,NB-063-001 't Hoefsven.jpg,None,51.6761,5.0942,1,15.0,46.0,"[5943930, 5943929, 5943892, 5943907, 5943911, ..."
1341,'t Nije Hemelriek,'t Nije Hemelriek,goed,De zwemlocatie “’t Nije Hemelriek” is gelegen ...,Recreatiezone_'t_Nije_Hemelriek.jpg,None,52.9725,6.7556,1,15.0,15.0,"[5943906, 5943913, 5943902, 5943930, 5943929, ..."
1315,'t Schaartven,'t Schaartven,goed,"‘t Schaartven is een voormalige zandwinplas, g...",None,None,51.5735,5.9749,1,15.0,15.0,"[5943892, 5943916, 5943897, 5943900, 5943929, ..."
1344,'t Veenmeer,'t Veenmeer,goed,De zwemlocatie ‟t Veenmeer is gelegen in Tynaa...,Recreatiezone_'t_Veenmeer.jpg,None,53.0829,6.6303,1,15.0,15.0,"[5943901, 5943913, 5943906, 5943900, 5943892, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...
7941840,Strand Kortgene Paardekreek,Paardekreek,WAARSCHUWING,Door camping de Paardekreek aangelegd zandstra...,None,None,51.5496,3.8096,1,15.0,15.0,NaN
7356682,Strand Zoutkamp,None,goed,strand zoutkamp in onderzoek,None,None,53.3388,6.2973,1,161.0,15.0,NaN
6837320,Strand Zuid Oldambtmeer,Strand Zuid,goed,strand zuid oldambtmeer,Recreatiezone Zuid Blauwe Stad kopie-1.jpg,None,53.1651,7.0372,2,253.0,46.0,"[5943927, 5943897, 5943892, 5943929, 5943916, ..."


### Waarschuwingen

In [141]:
preventative_measures = load_wfs_layer("zwr_public:zwemplek_maatregel")
preventative_measures = preventative_measures.drop(columns=['id', 'bw_code', 'object_type', 'toelichtingstandaardextended', 'type', 'geometry', 'zwemplek_naam'])
active_warnings = preventative_measures[(preventative_measures['objectbegintijd'] <= '2024-08-02')]
active_warnings = active_warnings.drop(columns=['redeningetrokken', 'objectbegintijd', 'objecteindtijd', 'omschrijving', 'key_id'])
active_warnings = active_warnings[[
    'zwemwaterlocatie_id', 'toelichtingstandaard', 'toelichtingpubliek']
]
active_warnings.set_index('zwemwaterlocatie_id', inplace=True)
active_warnings.rename(columns={
    'toelichtingstandaard': 'warning_type',
    'toelichtingpubliek': 'warning_description'
}, inplace=True)
active_warnings


,warning_type,warning_description
zwemwaterlocatie_id,,
1158,ONTRADEN_BLAUWALG,None
1092,ONTRADEN_KWALITEIT,None
1122,ONTRADEN_KWALITEIT,None
1243,WAARSCHUWING_BLAUWALG,None
1352,ONTRADEN_ZWEMMERSJEUK,None
...,...,...
964,WAARSCHUWING_BLAUWALG,None
1502,WAARSCHUWING_ONVEILIGE_SITUATIE,Gevaarlijke waterbodem
1264,WAARSCHUWING_BLAUWALG,None


In [142]:
warnings = active_warnings.groupby(['zwemwaterlocatie_id'])[['warning_type','warning_description']].apply(lambda x: x.to_dict('records')).reset_index(name='warnings')
warnings

,zwemwaterlocatie_id,warnings
0,960,"[{'warning_type': 'WAARSCHUWING_KWALITEIT', 'w..."
1,963,"[{'warning_type': 'WAARSCHUWING_ZWEMMERSJEUK',..."
2,964,"[{'warning_type': 'ONTRADEN_KWALITEIT', 'warni..."
3,965,"[{'warning_type': 'ONTRADEN_BLAUWALG', 'warnin..."
4,966,"[{'warning_type': 'WAARSCHUWING_BLAUWALG', 'wa..."
...,...,...
125,7356360,"[{'warning_type': 'WAARSCHUWING_ZWEMMERSJEUK',..."
126,7532190,"[{'warning_type': 'WAARSCHUWING_BLAUWALG', 'wa..."
127,7716690,"[{'warning_type': 'WAARSCHUWING_BLAUWALG', 'wa..."
128,7941840,[{'warning_type': 'WAARSCHUWING_BIJZONDERE_OMS...


In [143]:
data = data.merge(warnings, on="zwemwaterlocatie_id", how="left")
data

,zwemwaterlocatie_id,name,alternate_name,current_status,description,photos,website,lat,lon,eu_designation,e_coli,int_ent,voorzieningen,warnings
0,1323,Engelermeer Oost,Engelermeer Oost,goed,Het Engelermeer is een serie kunstmatige plass...,NB-031-001 Engelermeer Oost.jpg,None,51.7046,5.2461,1,61.0,30.0,"[5943902, 5943930, 5943929, 5943896, 5943923, ...",NaN
1,1259,'t Hoefsven,'t Hoefsven,goed,De locatie ligt ten zuidoosten van Waalwijk. E...,NB-063-001 't Hoefsven.jpg,None,51.6761,5.0942,1,15.0,46.0,"[5943930, 5943929, 5943892, 5943907, 5943911, ...",NaN
2,1341,'t Nije Hemelriek,'t Nije Hemelriek,goed,De zwemlocatie “’t Nije Hemelriek” is gelegen ...,Recreatiezone_'t_Nije_Hemelriek.jpg,None,52.9725,6.7556,1,15.0,15.0,"[5943906, 5943913, 5943902, 5943930, 5943929, ...",NaN
3,1315,'t Schaartven,'t Schaartven,goed,"‘t Schaartven is een voormalige zandwinplas, g...",None,None,51.5735,5.9749,1,15.0,15.0,"[5943892, 5943916, 5943897, 5943900, 5943929, ...",NaN
4,1344,'t Veenmeer,'t Veenmeer,goed,De zwemlocatie ‟t Veenmeer is gelegen in Tynaa...,Recreatiezone_'t_Veenmeer.jpg,None,53.0829,6.6303,1,15.0,15.0,"[5943901, 5943913, 5943906, 5943900, 5943892, ...",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
950,7941840,Strand Kortgene Paardekreek,Paardekreek,WAARSCHUWING,Door camping de Paardekreek aangelegd zandstra...,None,None,51.5496,3.8096,1,15.0,15.0,NaN,[{'warning_type': 'WAARSCHUWING_BIJZONDERE_OMS...
951,7356682,Strand Zoutkamp,None,goed,strand zoutkamp in onderzoek,None,None,53.3388,6.2973,1,161.0,15.0,NaN,NaN
952,6837320,Strand Zuid Oldambtmeer,Strand Zuid,goed,strand zuid oldambtmeer,Recreatiezone Zuid Blauwe Stad kopie-1.jpg,None,53.1651,7.0372,2,253.0,46.0,"[5943927, 5943897, 5943892, 5943929, 5943916, ...",NaN
953,6997562,Strandje Brouwershaven,Brouwershaven,goed,Strandje naast jachthaven van Brouwershaven,None,None,51.7315,3.9154,1,77.0,15.0,NaN,NaN


# GeoJSON

In [144]:
from geojson import Feature, Point, FeatureCollection

In [145]:
collection = []

In [146]:
for index, row in data.iterrows():
    print(row)
    feature = Feature(id=row['zwemwaterlocatie_id'],
                      geometry=Point((row['lat'], row['lon'])),
                      properties={
        "name": row['name'],
        "alternate_name": row['alternate_name']
    })
    collection.append(feature)


zwemwaterlocatie_id                                                 1323
name                                                    Engelermeer Oost
alternate_name                                          Engelermeer Oost
current_status                                                      goed
description            Het Engelermeer is een serie kunstmatige plass...
photos                                   NB-031-001 Engelermeer Oost.jpg
website                                                             None
lat                                                              51.7046
lon                                                               5.2461
eu_designation                                                         1
e_coli                                                              61.0
int_ent                                                             30.0
voorzieningen          [5943902, 5943930, 5943929, 5943896, 5943923, ...
warnings                                           

In [147]:
geojson = FeatureCollection(collection)


In [148]:
geojson

{"features": [{"geometry": {"coordinates": [51.7046, 5.2461], "type": "Point"}, "id": 1323, "properties": {"alternate_name": "Engelermeer Oost", "name": "Engelermeer Oost"}, "type": "Feature"}, {"geometry": {"coordinates": [51.6761, 5.0942], "type": "Point"}, "id": 1259, "properties": {"alternate_name": "'t Hoefsven", "name": "'t Hoefsven"}, "type": "Feature"}, {"geometry": {"coordinates": [52.9725, 6.7556], "type": "Point"}, "id": 1341, "properties": {"alternate_name": "'t Nije Hemelriek", "name": "'t Nije Hemelriek"}, "type": "Feature"}, {"geometry": {"coordinates": [51.5735, 5.9749], "type": "Point"}, "id": 1315, "properties": {"alternate_name": "'t Schaartven", "name": "'t Schaartven"}, "type": "Feature"}, {"geometry": {"coordinates": [53.0829, 6.6303], "type": "Point"}, "id": 1344, "properties": {"alternate_name": "'t Veenmeer", "name": "'t Veenmeer"}, "type": "Feature"}, {"geometry": {"coordinates": [51.3979, 5.4108], "type": "Point"}, "id": 1291, "properties": {"alternate_name":

In [150]:
GeoJSON(
    data=geojson
)

<IPython.display.GeoJSON object>